Six Nations stats = https://www.espn.co.uk/rugby/scoreboard/_/league/180659?date=20220205

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import selenium library and necessary features for web scraping
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [4]:
# settings for selenium
cService = webdriver.ChromeService(executable_path=r"C:\Users\jamie\Python Projects\Rugby\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=cService)

# open the six nations page for 6N23 04/02/23
driver.get("https://www.espn.co.uk/rugby/scoreboard/_/league/180659?date=20230204")

# close privacy settings window
time.sleep(5)
cookies_button = driver.find_element(By.ID, "onetrust-close-btn-container")
cookies_button.click()
time.sleep(3)

In [5]:
# navigate to first fixture page
fixtures_elements = driver.find_elements(By.CLASS_NAME, "competitors")

In [6]:
# open first fixture page
fixtures_elements[0].click()

In [7]:
# get url for current fixture
time.sleep(3)
get_url = driver.current_url
current_fixture_id = get_url.split("_")[1][1:].split("/")
match_stats_url = "https://www.espn.co.uk/rugby/matchstats?" + "gameId=" + current_fixture_id[1] + "&league=" + current_fixture_id[-1]
player_stats_url = "https://www.espn.co.uk/rugby/playerstats?" + "gameId=" + current_fixture_id[1] + "&league=" + current_fixture_id[-1]

## Match Stats Page
### Generic Stats
- Match date
- Team 1 and Team 2
- FT score
- Try scorers (Name & time scored)
- Conversion scorers (Name & time scored)
- Penalty scorers (Name & time scored)
- Drop goal scorers (Name & time scored)

In [9]:
# find date of match
date_string = driver.find_elements(By.CLASS_NAME, "game-date-time")
date_string = date_string[1].text

# convert to datetime object
match_datetime = pd.to_datetime(date_string, format="%I:%M %p, %B %d, %Y")
match_datetime

Timestamp('2023-02-04 14:15:00')

In [10]:
'''date_string_list = date_string.split()
match_year, match_month, match_day = int(date_string_list[4]), date_string_list[2], int(date_string_list[3].strip(","))
match_year, match_month, match_day'''

'date_string_list = date_string.split()\nmatch_year, match_month, match_day = int(date_string_list[4]), date_string_list[2], int(date_string_list[3].strip(","))\nmatch_year, match_month, match_day'

In [11]:
# normal driver
driver.get(match_stats_url)

# england v argentina world cup game (no tries or convs)
# driver.get("https://www.espn.co.uk/rugby/matchstats?gameId=596158&league=164205")

In [12]:
team_names = driver.find_elements(By.CLASS_NAME, "long-name")
team_names_list = [team_names[2].text, team_names[3].text]
team_names_list

['WALES', 'IRE']

In [13]:
scores = driver.find_elements(By.CLASS_NAME, "score-container")
scores_list = [scores[0].text, scores[1].text]
scores_list

['10', '34']

In [14]:
# ESPN's html has home and away team backwards
competitors_class = driver.find_elements(By.CLASS_NAME, "competitors")

competitors_class_list = []
for i in competitors_class:
    competitors_class_list.append(i.text)

In [15]:
# locate header score section
div_tag = competitors_class[1].find_elements(By.TAG_NAME, "div")
home_team_div_tag = div_tag[0]
away_team_div_tag = div_tag[2]

In [16]:
# find home team scorers, print "no scorers" for exceptions
try:
    home_team_tryscorers = home_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-solid-before")
    print(home_team_tryscorers.text)
except NoSuchElementException:
    home_team_tryscorers = 0
    
try:
    home_team_conv_scorers = home_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-conversion-solid-before")
    print(home_team_conv_scorers.text)
except NoSuchElementException:
    home_team_conv_scorers = 0
    
try:
    home_team_pen_scorers = home_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-penalty-solid-before")
    print(home_team_pen_scorers.text)
except NoSuchElementException:
    home_team_pen_scorers = 0

try:
    home_team_drop_scorers = home_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-drop-goal-solid-before")
    print(home_team_drop_scorers.text)
except NoSuchElementException:
    home_team_drop_scorers = 0

Liam Williams(46')
Dan Biggar(46')
Dan Biggar(14')


In [17]:
# find away team scorers, print "no scorers" for exceptions
try:
    away_team_tryscorers = away_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-solid-before")
    print(away_team_tryscorers.text)
except NoSuchElementException:
    away_team_tryscorers = 0

try:
    away_team_conv_scorers = away_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-conversion-solid-before")
    print(away_team_conv_scorers.text)
except NoSuchElementException:
    away_team_conv_scorers = 0
    
try:
    away_team_pen_scorers = away_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-penalty-solid-before")
    print(away_team_pen_scorers.text)
except NoSuchElementException:
    away_team_pen_scorers = 0
    
try:
    away_team_drop_scorers = away_team_div_tag.find_element(By.CLASS_NAME, "icon-rugby-drop-goal-solid-before")
    print(away_team_drop_scorers.text)
except NoSuchElementException:
    away_team_drop_scorers = 0

Caelan Doris(2')James Ryan(9')James Lowe(21')Josh van der Flier(73')
Johnny Sexton(3', 10', 22')Ross Byrne(73')
Johnny Sexton(19', 28')


In [18]:
# create dict of home tryscorers
# check if any tryscorers then proceed
if home_team_tryscorers != 0:
    home_tryscorers_split = home_team_tryscorers.text.split(")")


    home_tryscorers_clean = {}
    for i in range(len(home_tryscorers_split)-1):
        scorer = home_tryscorers_split[i].strip("'").split("(")
        home_tryscorers_clean[scorer[0]] = [int(scorer[1])]
else:
    home_tryscorers_clean = {}
print(home_tryscorers_clean)

{'Liam Williams': [46]}


In [19]:
# create dict of away tryscorers
# check if any tryscorers then proceed
if away_team_tryscorers != 0:
    away_tryscorers_split = away_team_tryscorers.text.split(")")

    away_tryscorers_clean = {}
    for i in range(len(away_tryscorers_split)-1):
        scorer = away_tryscorers_split[i].strip("'").split("(")
        away_tryscorers_clean[scorer[0]] = [int(scorer[1])]
else:
    away_tryscorers_clean = {}
print(away_tryscorers_clean)

{'Caelan Doris': [2], 'James Ryan': [9], 'James Lowe': [21], 'Josh van der Flier': [73]}


In [20]:
# create dict of home conversion scorers
# check if any tryscorers then proceed
if home_team_conv_scorers != 0:    
    home_conv_scorers_split = home_team_conv_scorers.text.split(")")

    home_conv_scorers_clean = []

    for i in home_conv_scorers_split:
        home_conv_scorers_clean.append(i.split("("))

    home_conv_scorers_cleaner = []

    for i in range(len(home_conv_scorers_clean)-1):
        conv_times = home_conv_scorers_clean[i][1].replace("'", "").split(",")
        home_conv_scorers_cleaner.append(conv_times)
    home_conv_scorers_cleaner

    home_conv_scorers_times = []

    for i in home_conv_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        home_conv_scorers_times.append(player_times)

    # create away conv scorers dict
    home_conv_scorers = {}
    for i in range(len(home_conv_scorers_clean)-1):
        home_conv_scorers[home_conv_scorers_clean[i][0]] = home_conv_scorers_times[i]
else:
    home_conv_scorers = {}
home_conv_scorers

{'Dan Biggar': [46]}

In [21]:
# create dict of away conversion scorers
# check if any tryscorers then proceed
if away_team_conv_scorers != 0: 
    away_conv_scorers_split = away_team_conv_scorers.text.split(")")

    away_conv_scorers_clean = []

    for i in away_conv_scorers_split:
        away_conv_scorers_clean.append(i.split("("))

    away_conv_scorers_cleaner = []

    for i in range(len(away_conv_scorers_clean)-1):
        conv_times = away_conv_scorers_clean[i][1].replace("'", "").split(",")
        away_conv_scorers_cleaner.append(conv_times)
    away_conv_scorers_cleaner

    away_conv_scorers_times = []

    for i in away_conv_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        away_conv_scorers_times.append(player_times)

    # create away conv scorers dict
    away_conv_scorers = {}
    for i in range(len(away_conv_scorers_clean)-1):
        away_conv_scorers[away_conv_scorers_clean[i][0]] = away_conv_scorers_times[i]
else:
    away_conv_scorers = {}
away_conv_scorers

{'Johnny Sexton': [3, 10, 22], 'Ross Byrne': [73]}

In [22]:
# create dict of home penalty scorers
# check if any tryscorers then proceed
if home_team_pen_scorers != 0: 
    home_pen_scorers_split = home_team_pen_scorers.text.split(")")

    home_pen_scorers_clean = []

    for i in home_pen_scorers_split:
        home_pen_scorers_clean.append(i.split("("))

    home_pen_scorers_cleaner = []

    for i in range(len(home_pen_scorers_clean)-1):
        conv_times = home_pen_scorers_clean[i][1].replace("'", "").split(",")
        home_pen_scorers_cleaner.append(conv_times)
    home_pen_scorers_cleaner

    home_pen_scorers_times = []

    for i in home_pen_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        home_pen_scorers_times.append(player_times)

    # create away conv scorers dict
    home_pen_scorers = {}
    for i in range(len(home_pen_scorers_clean)-1):
        home_pen_scorers[home_pen_scorers_clean[i][0]] = home_pen_scorers_times[i]
else:
    home_pen_scorers = {}
home_pen_scorers

{'Dan Biggar': [14]}

In [23]:
# create dict of away penalty scorers
# check if any tryscorers then proceed
if away_team_pen_scorers != 0: 
    away_pen_scorers_split = away_team_pen_scorers.text.split(")")

    away_pen_scorers_clean = []

    for i in away_pen_scorers_split:
        away_pen_scorers_clean.append(i.split("("))

    away_pen_scorers_cleaner = []

    for i in range(len(away_pen_scorers_clean)-1):
        conv_times = away_pen_scorers_clean[i][1].replace("'", "").split(",")
        away_pen_scorers_cleaner.append(conv_times)
    away_pen_scorers_cleaner

    away_pen_scorers_times = []

    for i in away_pen_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        away_pen_scorers_times.append(player_times)

    # create away conv scorers dict
    away_pen_scorers = {}
    for i in range(len(away_pen_scorers_clean)-1):
        away_pen_scorers[away_pen_scorers_clean[i][0]] = away_pen_scorers_times[i]
else:
    away_pen_scorers = {}
    
away_pen_scorers

{'Johnny Sexton': [19, 28]}

In [24]:
# create dict of home drop goal scorers
# check if any tryscorers then proceed
if home_team_drop_scorers != 0: 
    home_drop_scorers_split = home_team_drop_scorers.text.split(")")

    home_drop_scorers_clean = []

    for i in home_drop_scorers_split:
        home_drop_scorers_clean.append(i.split("("))

    home_drop_scorers_cleaner = []

    for i in range(len(home_drop_scorers_clean)-1):
        conv_times = home_drop_scorers_clean[i][1].replace("'", "").split(",")
        home_drop_scorers_cleaner.append(conv_times)
    home_drop_scorers_cleaner

    home_drop_scorers_times = []

    for i in home_drop_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        home_drop_scorers_times.append(player_times)

    # create away conv scorers dict
    home_drop_scorers = {}
    for i in range(len(home_drop_scorers_clean)-1):
        home_drop_scorers[home_drop_scorers_clean[i][0]] = home_drop_scorers_times[i]
else:
    home_drop_scorers = {}
home_drop_scorers

{}

In [25]:
# create dict of away drop goal scorers
# check if any tryscorers then proceed
if away_team_drop_scorers != 0: 
    away_drop_scorers_split = away_team_drop_scorers.text.split(")")

    away_drop_scorers_clean = []

    for i in away_drop_scorers_split:
        away_drop_scorers_clean.append(i.split("("))

    away_drop_scorers_cleaner = []

    for i in range(len(away_drop_scorers_clean)-1):
        conv_times = away_drop_scorers_clean[i][1].replace("'", "").split(",")
        away_drop_scorers_cleaner.append(conv_times)
    away_drop_scorers_cleaner

    away_drop_scorers_times = []

    for i in away_drop_scorers_cleaner:
        player_times = []
        for j in i:
            player_times.append(int(j))
        away_drop_scorers_times.append(player_times)

    # create away conv scorers dict
    away_drop_scorers = {}
    for i in range(len(away_drop_scorers_clean)-1):
        away_drop_scorers[away_drop_scorers_clean[i][0]] = away_drop_scorers_times[i]
else:
    away_drop_scorers = {}
away_drop_scorers

{}

### Match Events
Derive these totals from the Generic Stats
- No. Tries (len(tryscorers dict))
- No. Conversions
- No. Penalties

Scrape this
- Kick Percent Success

In [27]:
# match events
home_try_count = len(sum(home_tryscorers_clean.values(), []))
away_try_count = len(sum(away_tryscorers_clean.values(), []))

home_conv_count = len(sum(home_conv_scorers.values(), []))
away_conv_count = len(sum(away_conv_scorers.values(), []))

home_pen_goals_count = len(sum(home_pen_scorers.values(), []))
away_pen_goals_count = len(sum(away_pen_scorers.values(), []))

home_drop_goals_count = len(sum(home_drop_scorers.values(), []))
away_drop_goals_count = len(sum(away_drop_scorers.values(), []))

print(home_try_count, away_try_count)
print(home_conv_count, away_conv_count)
print(home_pen_goals_count, away_pen_goals_count)
print(home_drop_goals_count, away_drop_goals_count)

1 4
1 4
1 2
0 0


In [28]:
# non-graphic stats
non_graphic_stats_elements = driver.find_elements(By.CLASS_NAME, "compareTable")
for i in non_graphic_stats_elements:
    print(i.text)

1 Tries 4
1 Conversion Goals 4
1 Penalty Goals 2
100% Kick Percent Success 100%
25 Kicks From Hand 29
177 Passes 154
140 Runs 133
40% / 54% Possession 1H/2H 60% / 46%
46% / 51% Territory 1H/2H 54% / 49%
6 Clean Breaks 6
17 Defenders Beaten 33
10 Offload 5
98 / 102 (96%) Rucks Won 103 / 107 (96%)
3 / 5 (60%) Mauls Won 11 / 11 (100%)
14 Turnovers Conceded 8
0 Red Cards 0
1 Yellow Cards 0
1 Total Free Kicks Conceded 2


In [29]:
# kick percent
kick_perc_line = non_graphic_stats_elements[0].text.split("\n")[3].split()
home_kick_perc = int(kick_perc_line[0].split("%")[0])
away_kick_perc = int(kick_perc_line[-1].split("%")[0])
print(home_kick_perc, away_kick_perc)

100 100


### Kick/Pass/Run
- Metres Run
- Kicks from hand
- Passes
- Runs

In [31]:
# line graph elements
line_graphs = driver.find_elements(By.CLASS_NAME, "compareLineGraph")
metres_run_graph = line_graphs[0]
total_possession_graph = line_graphs[1]
total_territory_graph = line_graphs[2]

In [32]:
# metres run
home_metres_run = int(metres_run_graph.text.split("\n")[0])
away_metres_run = int(metres_run_graph.text.split("\n")[1])
home_metres_run, away_metres_run

(421, 436)

In [33]:
# kicks from hand
kicks_hand_line = non_graphic_stats_elements[1].text.split("\n")[0]
home_kicks_hand = int(kicks_hand_line.split()[0])
away_kicks_hand = int(kicks_hand_line.split()[-1])
home_kicks_hand, away_kicks_hand

(25, 29)

In [34]:
# passes
passes_line = non_graphic_stats_elements[1].text.split("\n")[1]
home_passes = int(passes_line.split()[0])
away_passes = int(passes_line.split()[-1])
home_passes, away_passes

(177, 154)

In [35]:
# runs
runs_line = non_graphic_stats_elements[1].text.split("\n")[2]
home_runs = int(runs_line.split()[0])
away_runs = int(runs_line.split()[-1])
home_runs, away_runs

(140, 133)

### Attacking
- Possession 1H/2H
- Territory 1H/2H
- Clean Breaks
- Defenders Beaten
- Offload
- Rucks Won (raw & percent)
- Mauls Won (raw & percent)
- Turnovers Conceded

In [37]:
# 1st and 2nd half possession
possession_line = non_graphic_stats_elements[2].text.split("\n")[0].split()
home_half1_posession = int(possession_line[0].split("%")[0])
away_half1_posession = int(possession_line[5].split("%")[0])

home_half2_posession = int(possession_line[2].split("%")[0])
away_half2_posession = int(possession_line[-1].split("%")[0])

home_half1_posession, away_half1_posession, home_half2_posession, away_half2_posession

(40, 60, 54, 46)

In [38]:
# 1st and 2nd half territory
territory_line = non_graphic_stats_elements[2].text.split("\n")[1].split()

home_half1_territory = int(territory_line[0].split("%")[0])
away_half1_territory = int(territory_line[5].split("%")[0])

home_half2_territory = int(territory_line[2].split("%")[0])
away_half2_territory = int(territory_line[-1].split("%")[0])

home_half1_territory, away_half1_territory, home_half2_territory, away_half2_territory

(46, 54, 51, 49)

In [39]:
# clean breaks
clean_break_line = non_graphic_stats_elements[2].text.split("\n")[2].split()
home_clean_breaks = int(clean_break_line[0])
away_clean_breaks = int(clean_break_line[-1])

home_clean_breaks, away_clean_breaks

(6, 6)

In [40]:
# defenders beaten
def_beaten_line = non_graphic_stats_elements[2].text.split("\n")[3].split()
home_def_beaten = int(def_beaten_line[0])
away_def_beaten = int(def_beaten_line[-1])

home_def_beaten, away_def_beaten

(17, 33)

In [41]:
# offloads
offloads_line = non_graphic_stats_elements[2].text.split("\n")[4].split()
home_offloads = int(offloads_line[0])
away_offloads = int(offloads_line[-1])

home_offloads, away_offloads

(10, 5)

In [42]:
# rucks won (raw & percent)
rucks_won_line = non_graphic_stats_elements[2].text.split("\n")[5].split()

home_rucks_won = int(rucks_won_line[0])
home_rucks_lost = int(rucks_won_line[2]) - home_rucks_won
home_rucks_perc = int(rucks_won_line[3].split("%")[0].split("(")[1])

away_rucks_won = int(rucks_won_line[6])
away_rucks_lost = int(rucks_won_line[8]) - away_rucks_won
away_rucks_perc = int(rucks_won_line[-1].split("%")[0].split("(")[1])

home_rucks_won, home_rucks_lost, home_rucks_perc, away_rucks_won, away_rucks_lost, away_rucks_perc

(98, 4, 96, 103, 4, 96)

In [43]:
# mauls won (raw & percent)
mauls_won_line = non_graphic_stats_elements[2].text.split("\n")[6].split()

home_mauls_won = int(mauls_won_line[0])
home_mauls_lost = int(mauls_won_line[2]) - home_mauls_won
home_mauls_perc = int(mauls_won_line[3].split("%")[0].split("(")[1])

away_mauls_won = int(mauls_won_line[6])
away_mauls_lost = int(mauls_won_line[8]) - away_mauls_won
away_mauls_perc = int(mauls_won_line[-1].split("%")[0].split("(")[1])

home_mauls_won, home_mauls_lost, home_mauls_perc, away_mauls_won, away_mauls_lost, away_mauls_perc

(3, 2, 60, 11, 0, 100)

In [44]:
# turnovers conceded
turnovers_line = non_graphic_stats_elements[2].text.split("\n")[7].split()
home_turnovers_conc = int(turnovers_line[0])
away_turnovers_conc = int(turnovers_line[-1])

home_turnovers_conc, away_turnovers_conc

(14, 8)

### Total Possession and Territory

In [46]:
# total possession
home_total_possession = int(total_possession_graph.text.split("%")[0])
away_total_possession = int(total_possession_graph.text.split("%")[-2])

home_total_possession, away_total_possession

(48, 52)

In [47]:
# total territory
home_total_territory = int(total_territory_graph.text.split("%")[0])
away_total_territory = int(total_territory_graph.text.split("%")[-2])

home_total_territory, away_total_territory

(49, 51)

### Set Pieces
- Scrums won (raw & percent)
- Lineouts won (raw & percent)

In [49]:
# count charts
count_charts = driver.find_elements(By.CLASS_NAME, "countChart")
scrums_won_chart = count_charts[0].text
lineouts_won_chart = count_charts[1].text
penalties_conceded_chart = count_charts[2].text

In [50]:
# scrums won (raw & percent)
home_scrums_won = int(scrums_won_chart.split("\n")[0].split("/")[0])
home_scrums_lost = int(scrums_won_chart.split("\n")[0].split("/")[1].split()[0]) - home_scrums_won
home_scrums_perc = int(scrums_won_chart.split("%")[0].split("(")[1])

away_scrums_won = int(scrums_won_chart.split("\n")[1].split("/")[0])
away_scrums_lost = int(scrums_won_chart.split("\n")[1].split("/")[1].split()[0]) - away_scrums_won
away_scrums_perc = int(scrums_won_chart.split("%")[1].split("(")[1])

home_scrums_won, home_scrums_lost, home_scrums_perc, away_scrums_won, away_scrums_lost, away_scrums_perc

(3, 1, 75, 6, 3, 66)

In [51]:
# lineouts won (raw & percent)
home_lineouts_won = int(lineouts_won_chart.split("\n")[0].split("/")[0])
home_lineouts_lost = int(lineouts_won_chart.split("\n")[0].split("/")[1].split()[0]) - home_lineouts_won
home_lineouts_perc = int(lineouts_won_chart.split("%")[0].split("(")[1])

away_lineouts_won = int(lineouts_won_chart.split("\n")[1].split("/")[0])
away_lineouts_lost = int(lineouts_won_chart.split("\n")[1].split("/")[1].split()[0]) - away_lineouts_won
away_lineouts_perc = int(lineouts_won_chart.split("%")[1].split("(")[1])

home_lineouts_won, home_lineouts_lost, home_lineouts_perc, away_lineouts_won, away_lineouts_lost, away_lineouts_perc

(10, 3, 76, 15, 0, 100)

### Defending
- Tackles (raw & percent)

In [53]:
# find tackle chart elements
tackle_charts = driver.find_elements(By.CLASS_NAME, "inner-circle")

In [54]:
# home and away tackles (raw & percent)
home_tackles = int(tackle_charts[0].text.split("\n")[0].split("/")[0])
home_missed_tackles = int(tackle_charts[0].text.split("\n")[0].split("/")[1]) - home_tackles
home_tackle_perc = int(tackle_charts[0].text.split("\n")[1].split("%")[0])

away_tackles = int(tackle_charts[1].text.split("\n")[0].split("/")[0])
away_missed_tackles = int(tackle_charts[1].text.split("\n")[0].split("/")[1]) - away_tackles
away_tackle_perc = int(tackle_charts[1].text.split("\n")[1].split("%")[0])

home_tackles, home_missed_tackles, home_tackle_perc, away_tackles, away_missed_tackles, away_tackle_perc

(184, 33, 85, 155, 17, 90)

### Discipline
- Red Cards
- Yellow cards
- Free kicks conceded
- Penalties conceded

In [56]:
# red cards
red_cards_line = non_graphic_stats_elements[3].text.split("\n")[0].split()
home_red_cards = int(red_cards_line[0])
away_red_cards = int(red_cards_line[-1])
home_red_cards, away_red_cards

(0, 0)

In [57]:
# yellow cards
yellow_cards_line = non_graphic_stats_elements[3].text.split("\n")[1].split()
home_yellow_cards = int(yellow_cards_line[0])
away_yellow_cards = int(yellow_cards_line[-1])
home_yellow_cards, away_yellow_cards

(1, 0)

In [58]:
# free kicks
freekicks_line = non_graphic_stats_elements[3].text.split("\n")[2].split()
home_freekicks_conceded = int(freekicks_line[0])
away_freekicks_conceded = int(freekicks_line[-1])
home_freekicks_conceded, away_freekicks_conceded

(1, 2)

In [59]:
# penalties conceded
home_pens_conceded = int(penalties_conceded_chart.replace("\n", "").split("<")[0])
away_pens_conceded = int(penalties_conceded_chart.replace("\n", "").split("<")[home_pens_conceded])
home_pens_conceded, away_pens_conceded

(15, 13)

## Turn Stats into DataFrame

In [61]:
# create dataframes for one match - one df per stats category with MatchID as primary/foreign key
matchID = 0
match_events_row = {"MatchID": matchID,
                    "Date": match_datetime,
                    "home_team": team_names_list[0],
                    "away_team": team_names_list[1],
                    "home score": scores_list[0],
                    "away score": scores_list[1],
                    "home_tries": home_try_count,
                    "away_tries": away_try_count,
                    "home_conv": home_conv_count,
                    "away_conv": away_conv_count,
                    "home_pen_goals": home_pen_goals_count,
                    "away_pen_goals": away_pen_goals_count,
                    "home_drops": home_drop_goals_count,
                    "away_drops": away_drop_goals_count,
                    "home_kick_success": home_kick_perc,
                    "away_kick_success": away_kick_perc,
                    "home_tryscorers": [home_tryscorers_clean],
                    "away_tryscorers": [away_tryscorers_clean],
                    "home_conv_scorers": [home_conv_scorers],
                    "away_conv_scorers": [away_conv_scorers],
                    "home_pen_scorers": [home_pen_scorers],
                    "away_pen_scorers": [away_pen_scorers],
                    "home_drop_scorers": [home_drop_scorers],
                    "away_drop_scorers": [away_drop_scorers]}

kick_pass_run_row = {"MatchID": matchID,
                    "home_m_run": home_metres_run,
                    "away_m_run": away_metres_run,
                    "home_kicks_hand": home_kicks_hand,
                    "away_kicks_hand": away_kicks_hand,
                    "home_passes": home_passes,
                    "away_passes": away_passes,
                    "home_runs": home_runs,
                    "away_runs": away_runs}

attacking_row = {"MatchID": matchID,
                "home_total_poss": home_total_possession,
                "away_total_poss": away_total_possession,
                "home_h1_poss": home_half1_posession,
                "away_h1_poss": away_half1_posession,
                "home_h2_poss": home_half2_posession,
                "away_h2_poss": away_half2_posession,
                "home_total_terr": home_total_territory,
                "away_total_terr": away_total_territory,
                "home_h1_terr": home_half1_territory,
                "away_h1_terr": away_half1_territory,
                "home_h2_terr": home_half2_territory,
                "away_h2_terr": away_half2_territory,
                "home_clean_br": home_clean_breaks,
                "away_clean_br": away_clean_breaks,
                "home_def_beat": home_def_beaten,
                "away_def_beat": away_def_beaten,
                "home_offl": home_offloads,
                "away_offl": away_offloads}

breakdown_row = {"MatchID": matchID,
                "home_rucks_won": home_rucks_won,
                "away_rucks_won": away_rucks_won,
                "home_rucks_lost": home_rucks_lost,
                "away_rucks_lost": away_rucks_lost,
                "home_rucks_perc": home_rucks_perc,
                "away_rucks_perc": away_rucks_perc,
                "home_mauls_won": home_mauls_won,
                "away_mauls_won": away_mauls_won,
                "home_mauls_lost": home_mauls_lost,
                "away_mauls_lost": away_mauls_lost,
                "home_mauls_perc": home_mauls_perc,
                "away_mauls_perc": away_mauls_perc,
                "home_turn_conc": home_turnovers_conc,
                "away_turn_conc": away_turnovers_conc}
                

In [62]:
defending_row = {"MatchID": matchID,
                "home_tackles": home_tackles,
                "away_tackles": away_tackles,
                "home_miss_tack": home_missed_tackles,
                "away_miss_tack": away_missed_tackles,
                "home_tack_perc": home_tackle_perc,
                "away_tack_perc": away_tackle_perc}

set_pieces_row = {"MatchID": matchID,
                "home_scrum_won": home_scrums_won,
                "away_scrum_won": away_scrums_won,
                "home_scrum_lost": home_scrums_lost,
                "away_scrum_lost": away_scrums_lost,
                "home_scrum_perc": home_scrums_perc,
                "away_scrum_perc": away_scrums_perc,
                "home_line_won": home_lineouts_won,
                "away_line_won": away_lineouts_won,
                "home_line_lost": home_lineouts_lost,
                "away_line_lost": away_lineouts_lost,
                "home_line_perc": home_lineouts_perc,
                "away_line_perc": away_lineouts_perc}

discipline_row = {"MatchID": matchID,
                "home_pens_conc": home_pens_conceded,
                "away_pens_conc": away_pens_conceded,
                "home_free_conc": home_freekicks_conceded,
                "away_free_conc": away_freekicks_conceded,
                "home_yellow": home_yellow_cards,
                "away_yellow": away_yellow_cards,
                "home_red": home_red_cards,
                "away_red": away_red_cards}

In [63]:
# create dfs for each new row
match_events_df = pd.DataFrame(match_events_row, index=[0])
kick_pass_run_df = pd.DataFrame(kick_pass_run_row, index=[0])
attacking_df = pd.DataFrame(attacking_row, index=[0])
breakdown_df = pd.DataFrame(breakdown_row, index=[0])
defending_df = pd.DataFrame(defending_row, index=[0])
set_pieces_df = pd.DataFrame(set_pieces_row, index=[0])
discipline_df = pd.DataFrame(discipline_row, index=[0])

In [141]:
match_events_df["Date"][0].date

<bound method Timestamp.date of Timestamp('2023-02-04 14:15:00')>

In [65]:
match_events_df["away_tryscorers"][0].keys()

dict_keys(['Caelan Doris', 'James Ryan', 'James Lowe', 'Josh van der Flier'])

### Head to Head Record
- last five games
### Form
- Team 1 form
- team 2 form